In [1]:
import numpy as np
import feather as ftr
import gc
import os
import logging
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
warnings.filterwarnings('ignore')

In [ ]:
def augmation(x,y,t=2):
    xs,xn = [],[]
    for i in range(t):
        mask = y>0
        
        #targetが1のものだけコピー
        x1 = x[mask].copy()
        print(x1.shape)
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xs.append(x1)

    for i in range(t//2):
        mask = y==0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        print(x1.shape)
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xn.append(x1)
    print('xs')
    print(len(xs))
    xs = np.vstack(xs)
    print(len(xs))
    print('xn')
    print(len(xn))
    xn = np.vstack(xn)
    print(len(xn))
    ys = np.ones(xs.shape[0])
    yn = np.zeros(xn.shape[0])
    x = np.vstack([x,xs,xn])
    y = np.concatenate([y,ys,yn])
    return x,y

In [ ]:
train = ftr.read_dataframe('../data/input/train.feather')
test = ftr.read_dataframe('../data/input/test.feather')

In [ ]:
features = [col for col in train.columns if col not in ['target', 'ID_code']]
X_train, y_train = train.iloc[:][features], train.iloc[:]['target']


In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
random_state=42
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
for fold, (trn_idx, val_idx) in enumerate(skf.split(train, train['target'])):
    X_train, y_train = train.iloc[trn_idx][features], train.iloc[trn_idx]['target']
    X_valid, y_valid = train.iloc[val_idx][features], train.iloc[val_idx]['target']
    augmation(X_train.values,y_train.values,t=2)